# Turbos validation working file



### configuration (yaml) file and the necessary library for this validation:

In [19]:
import pandas as pd
import numpy as np
import json
import yaml
import websocket
import math
import time
import random
import ast


SECONDS_IN_A_YEAR = 31536000

import warnings
warnings.filterwarnings("ignore")


In [20]:
asset = pd.read_csv("assets.csv")
asset.drop(["Leverage","Lot size","Min Lot size","Expected Price Change - 2sec","Expected Price Change (%) - 2sec"], axis='columns', inplace=True)
asset["pip_size"] = 1/10**asset["Decimal Places"]
asset

,Asset,IV,Tick Size,Min Stake,Max Stake,Decimal Places,TCUT,TCUI,TCUD,TCDT,TCDI,TCDD,Min Mul,Max Mul,Nopen,n,t,pip_size
0,R_10,0.10,0.000025,1,2000,3,0.9,0.9,0.9,0.9,0.9,0.9,100,1000,5,20,300,0.0010
1,R_25,0.25,0.000063,1,2000,3,0.9,0.9,0.9,0.9,0.9,0.9,50,500,5,20,300,0.0010
2,R_50,0.50,0.000126,1,2000,4,0.9,0.9,0.9,0.9,0.9,0.9,20,200,5,20,300,0.0001
3,R_75,0.75,0.000189,1,2000,4,1.1,1.1,1.1,1.1,1.1,1.1,15,150,5,20,300,0.0001
4,R_100,1.00,0.000252,1,2000,2,0.9,0.9,0.9,0.9,0.9,0.9,10,100,5,20,300,0.0100
5,1HZ10V,0.10,0.000018,1,2000,2,0.9,0.9,0.9,0.9,0.9,0.9,100,1000,5,20,300,0.0100
6,1HZ25V,0.25,0.000044,1,2000,2,0.9,0.9,0.9,0.9,0.9,0.9,50,500,5,20,300,0.0100
7,1HZ50V,0.50,0.000089,1,2000,2,0.9,0.9,0.9,0.9,0.9,0.9,20,200,5,20,300,0.0100
8,1HZ75V,0.75,0.000134,1,2000,2,0.9,0.9,0.9,0.9,0.9,0.9,15,150,5,20,300,0.0100
9,1HZ100V,1.00,0.000178,1,2000,2,0.9,0.9,0.9,0.9,0.9,0.9,10,100,5,20,300,0.0100


## Price proposal


In [168]:
api_token = "insert_api_token_here"
ws = "wss://qa69.deriv.dev/websockets/v3?l=EN&app_id=16303&brand=deriv"  
# "wss://ws.binaryws.com/websockets/v3?l=EN&app_id=1089"


connection = websocket.create_connection(ws)

def get_rand_duration(duration_unit): #return random number based on duration unit

    if duration_unit == 'd':
        max_duration = 365
        min_duration  = 1
    elif duration_unit == 'm':   
        max_duration = 1440
        min_duration = 1
    elif duration_unit == 's':   
        max_duration = 86400
        min_duration = 15
    elif duration_unit == 'h':   
        max_duration = 24
        min_duration = 1
    elif duration_unit == 't':   
        max_duration = 10
        min_duration = 5

    return np.random.randint(min_duration,max_duration)

def auth_token(api_token):   
    request_authorize = {
    "authorize": f"{api_token}"
    }
    connection.send(json.dumps(request_authorize))

    res = connection.recv()
    if res is None:
        print("error")
    data = json.loads(res)
    if "error" in data:
        print("Error:", data["error"]["message"])
        connection.close()
        
    elif data["msg_type"] == "authorize":
        print(data["authorize"])

def get_barrier_valid(amount, symbol, contract_type, duration_unit): #will get error response, and return the first valid barrier
    
    request = {
                "proposal": 1,
                "amount": amount,
                "barrier": "+0.0000001",
                "basis": "stake",
                "contract_type": contract_type,
                "currency": "USD",
                "duration": get_rand_duration(duration_unit),
                "duration_unit": duration_unit,
                "symbol": symbol
            }
    
    df = pd.DataFrame(columns=["spot", "spot_time", "last_tick_epoch", "high_barrier", "low_barrier"])

    connection.send(json.dumps(request))
    while True:
        res = connection.recv()
        if res is None:
            break
        data = json.loads(res)
        if data["msg_type"] == "proposal":
            if 'details' in data['error']:
                return data["error"]["details"]["barrier_choices"][1]
            else:
                return "0.0001"

def get_barrier_choices(amount, symbol, contract_type, duration_unit): #use to get, the barrier choices, spot price, etc
    barriers = get_barrier_valid(amount, symbol, contract_type, duration_unit)
    request = {
                "proposal": 1,
                "amount": amount,
                "barrier": barriers,
                "basis": "stake",
                "contract_type": contract_type,
                "currency": "USD",
                "duration": get_rand_duration(duration_unit),
                "duration_unit": duration_unit,
                "symbol": symbol
            }
    
    df = pd.DataFrame(columns=["symbol", "spot", "duration", "duration_unit", "contract_type", "barrier_choices"])

    connection.send(json.dumps(request))
    while True:
        res = connection.recv()
        if res is None:
            break
        data = json.loads(res)
        if data["msg_type"] == "proposal":
            if "proposal" in data:
                barrier_data = data["proposal"]["barrier_choices"] 
                response_data = {
                    "symbol": data["echo_req"]["symbol"],
                    "spot": data["proposal"]["spot"],
                    "duration": data["echo_req"]["duration"],
                    "duration_unit": data["echo_req"]["duration_unit"],
                    "contract_type": data["echo_req"]["contract_type"],
                    "barrier_choices": f"{barrier_data}",
                    "min_stake": data["proposal"]["min_stake"],
                    "max_stake": data["proposal"]["max_stake"],
                    "number_of_contracts": data["proposal"]["number_of_contracts"],
                    "date_expiry": data["proposal"]["date_expiry"],
                    "date_start": data["proposal"]["date_start"],
                    "payout": data["proposal"]["payout"],
                    "stake": amount,
                    
                }
            else: 
                msg = data["error"]["message"] 
                response_data = {
                    "stake": amount,
                    "message": msg,
                    "symbol": data["echo_req"]["symbol"],
                    "duration": data["echo_req"]["duration"],
                    "duration_unit": data["echo_req"]["duration_unit"],
                    "contract_type": data["echo_req"]["contract_type"],
                }
            
            df = pd.concat([df, pd.DataFrame(response_data, index=[0])])
            return df


def strike_price_choices(ul,current_spot,contract_types): #calculate barrier choices, refer perl or be specs 

    n_max = float(asset[asset["Asset"] == f"{ul}"]["Max Stake"] * asset[asset["Asset"] == f"{ul}"]["Max Mul"])
    sigma = float(asset[asset["Asset"]== f"{ul}"]["IV"])
    t = int(asset[asset["Asset"]== f"{ul}"]["t"])
    n = int(asset[asset["Asset"]== f"{ul}"]["n"]) -1
    sentiment = np.where(contract_types == "TURBOSLONG", "up", "down")

    alpha = ((0.5 / max((sigma * math.sqrt(t / SECONDS_IN_A_YEAR)), (1 / (n_max * current_spot))))**(1 / n)) - 1
    beta = max((sigma * math.sqrt(t / SECONDS_IN_A_YEAR)), (1 / (n_max * current_spot)))

    strike_price_choices = []
    for i in range(n + 1):
        distance_factor = beta * ((1 + alpha)**i)
        distance_relative = current_spot * distance_factor
        distance_factor = round_up(distance_relative, float(asset[asset["Asset"] == f"{ul}"]["pip_size"]))
        distance_factor = f"-{distance_factor}" if sentiment == 'up' else f"+{distance_factor}"
        strike_price_choices.append(distance_factor)

    return strike_price_choices

def round_up(value, pip_size):
    return math.ceil(value / pip_size) * pip_size


In [311]:
contract_types = ["TURBOSLONG","TURBOSSHORT"]
indices = ["R_10","R_25","R_50","R_75","R_100","1HZ10V","1HZ25V","1HZ50V","1HZ75V","1HZ100V"]
amount = np.arange(1,10,1).tolist()
duration_units = ["d","m","s","h","t"] #Duration unit - s: seconds, m: minutes, h: hours, d: days, t: ticks.


In [92]:
df = pd.DataFrame(columns=["symbol", "spot", "duration", "duration_unit", "contract_type", "stake"])

for i in range(200):
    connection = websocket.create_connection(ws)
    auth_token(api_token)
    stake = random.choice(amount)
    contract_type = random.choice(contract_types)
    duration_unit = random.choice(duration_units)
    underlying = random.choice(indices)

    df = pd.concat([df, get_barrier_choices(stake, underlying, contract_type, duration_unit)], ignore_index=True)
    connection.close()

df.to_csv("save_test_cases_price_proposal.csv") 

{'account_list': [{'account_category': 'trading', 'account_type': 'binary', 'created_at': 1694504665, 'currency': 'USD', 'is_disabled': 0, 'is_virtual': 0, 'landing_company_name': 'svg', 'linked_to': [], 'loginid': 'CR90000011'}, {'account_category': 'trading', 'account_type': 'binary', 'created_at': 1694504665, 'currency': 'USD', 'is_disabled': 0, 'is_virtual': 1, 'landing_company_name': 'virtual', 'linked_to': [], 'loginid': 'VRTC90000011'}], 'balance': 10000, 'country': 'aq', 'currency': 'USD', 'email': 'onea@gaawd.com', 'fullname': '  ', 'is_virtual': 1, 'landing_company_fullname': 'Deriv Limited', 'landing_company_name': 'virtual', 'linked_to': [], 'local_currencies': {'AAD': {'fractional_digits': 2}}, 'loginid': 'VRTC90000011', 'preferred_language': 'EN', 'scopes': ['read', 'trade', 'payments', 'admin'], 'upgradeable_landing_companies': ['svg'], 'user_id': 12}
{'account_list': [{'account_category': 'trading', 'account_type': 'binary', 'created_at': 1694504665, 'currency': 'USD', 

# Validate the Price Proposal
    - Barrier Choices
    - Duration
    - Min/Max Stake
    - Deriv's Comission

In [138]:
df = pd.read_csv("turbos_pp_test_cases_2.csv")

# Drop rows where 'message' column is not equal to NaN
df = df[df['message'].isna()]

# Drop the 'Unnamed: 0' column
df = df.drop('Unnamed: 0', axis=1)

# Reset the index if desired
df = df.reset_index(drop=True)

df_barrier = df.copy()
df_duration = df.copy()
df_stake = df.copy()
df_ppp = df.copy()

df_barrier.drop(["duration", "duration_unit", "stake", "number_of_contracts","date_expiry","date_start" ,'payout','message','min_stake','max_stake'], axis='columns', inplace=True)
df_duration.drop(['barrier_choices',"stake", "number_of_contracts",'payout','message','min_stake','max_stake'], axis='columns', inplace=True)
df_stake.drop([ "number_of_contracts",'payout','message','date_expiry', 'date_start'], axis='columns', inplace=True)
df_ppp.drop(['payout','message','date_expiry', 'date_start'], axis='columns', inplace=True)



## Validate the barrier choices

### define all the functions needed

In [139]:
def strike_price_choices(ul,current_spot,contract_types): #calculate barrier choices, refer perl or BE specs 

    n_max = float(asset[asset["Asset"] == f"{ul}"]["Max Stake"] * asset[asset["Asset"] == f"{ul}"]["Max Mul"])
    sigma = float(asset[asset["Asset"]== f"{ul}"]["IV"])
    t = int(asset[asset["Asset"]== f"{ul}"]["t"])
    n = int(asset[asset["Asset"]== f"{ul}"]["n"]) - 1
    sentiment = np.where(contract_types == "TURBOSLONG", "up", "down")

    alpha = ((0.5 / max((sigma * math.sqrt(t / SECONDS_IN_A_YEAR)), (1 / (n_max * current_spot))))**(1 / n)) - 1
    beta = max((sigma * math.sqrt(t / SECONDS_IN_A_YEAR)), (1 / (n_max * current_spot)))

    strike_price_choices = []
    for i in range(n + 1):
        distance_factor = beta * ((1 + alpha)**i)
        distance_relative = current_spot * distance_factor
        # print(distance_relative, float(asset[asset["Asset"] == f"{ul}"]["pip_size"]))
        distance_factor = round_up(distance_relative, float(asset[asset["Asset"] == f"{ul}"]["pip_size"]))
        distance_factor = f"-{distance_factor}" if sentiment == 'up' else f"+{distance_factor}"
        strike_price_choices.append(distance_factor)

    return strike_price_choices

def round_up(value, pip_size):
    return math.ceil(value / pip_size) * pip_size

def bs_price(type_of_contract, S, K):
    if type_of_contract == "TURBOSLONG":
        return S-K
    else:
        return K-S
    

def up_strike_price_choices(ul,current_spot,contract_types): #calculate upper threshold of the barrier choices, refer perl or BE specs 

    n_max = float(asset[asset["Asset"] == f"{ul}"]["Max Stake"] * asset[asset["Asset"] == f"{ul}"]["Max Mul"])
    sigma = float(asset[asset["Asset"]== f"{ul}"]["IV"])
    t = int(asset[asset["Asset"]== f"{ul}"]["t"])
    n = int(asset[asset["Asset"]== f"{ul}"]["n"]) -1
    sentiment = np.where(contract_types == "TURBOSLONG", "up", "down")

    alpha = ((0.5 / max((sigma * math.sqrt(t / SECONDS_IN_A_YEAR)), (1 / (n_max * current_spot))))**(1 / n)) - 1
    beta = max((sigma * math.sqrt(t / SECONDS_IN_A_YEAR)), (1 / (n_max * current_spot)))

    strike_price_choices = []
    for i in range(n + 1):
        distance_factor = beta * ((1 + alpha)**i)
        distance_relative = (current_spot*(1 + float(asset[asset["Asset"] == f"{ul}"]["IV"])*np.sqrt(7200/365/24/60/60))) * distance_factor
        # print(distance_relative, float(asset[asset["Asset"] == f"{ul}"]["pip_size"]))
        distance_factor = round_up(distance_relative, float(asset[asset["Asset"] == f"{ul}"]["pip_size"]))
        distance_factor = f"-{distance_factor}" if sentiment == 'up' else f"+{distance_factor}"
        strike_price_choices.append(distance_factor)

    return strike_price_choices

def low_strike_price_choices(ul,current_spot,contract_types): #calculate lower threshold of barrier choices, refer perl or BE specs 

    n_max = float(asset[asset["Asset"] == f"{ul}"]["Max Stake"] * asset[asset["Asset"] == f"{ul}"]["Max Mul"])
    sigma = float(asset[asset["Asset"]== f"{ul}"]["IV"])
    t = int(asset[asset["Asset"]== f"{ul}"]["t"])
    n = int(asset[asset["Asset"]== f"{ul}"]["n"]) -1
    sentiment = np.where(contract_types == "TURBOSLONG", "up", "down")

    alpha = ((0.5 / max((sigma * math.sqrt(t / SECONDS_IN_A_YEAR)), (1 / (n_max * current_spot))))**(1 / n)) - 1
    beta = max((sigma * math.sqrt(t / SECONDS_IN_A_YEAR)), (1 / (n_max * current_spot)))

    strike_price_choices = []
    for i in range(n + 1):
        distance_factor = beta * ((1 + alpha)**i)
        distance_relative = (current_spot* (1 - float(asset[asset["Asset"] == f"{ul}"]["IV"])*np.sqrt(7200/365/24/60/60))) * distance_factor
        # print(distance_relative, float(asset[asset["Asset"] == f"{ul}"]["pip_size"]))
        distance_factor = round_up(distance_relative, float(asset[asset["Asset"] == f"{ul}"]["pip_size"]))
        distance_factor = f"-{distance_factor}" if sentiment == 'up' else f"+{distance_factor}"
        strike_price_choices.append(distance_factor)

    return strike_price_choices

In [140]:
barriers_array = []
up_barriers_array = []
low_barriers_array = []

for row in df_barrier.iterrows():
    print (str(df_barrier["symbol"][row[0]]), float(df_barrier["spot"][row[0]]), str(df_barrier["contract_type"][row[0]]))
    barriers = strike_price_choices(str(df_barrier["symbol"][row[0]]), float(df_barrier["spot"][row[0]]), str(df_barrier["contract_type"][row[0]]))
    up_barriers = up_strike_price_choices(str(df_barrier["symbol"][row[0]]), float(df_barrier["spot"][row[0]]), str(df_barrier["contract_type"][row[0]]))
    low_barriers = low_strike_price_choices(str(df_barrier["symbol"][row[0]]), float(df_barrier["spot"][row[0]]), str(df_barrier["contract_type"][row[0]]))

    barriers_array.append(str(barriers))
    up_barriers_array.append(str(up_barriers))
    low_barriers_array.append(str(low_barriers))


df_barrier["calc_barrier_choices"] = barriers_array
df_barrier["up_barriers"] = up_barriers_array
df_barrier["low_barriers"] = low_barriers_array

R_10 9989.746 TURBOSSHORT
R_25 10079.959 TURBOSLONG
R_50 10056.7846 TURBOSSHORT
1HZ100V 10420.14 TURBOSLONG
1HZ10V 9999.79 TURBOSLONG
1HZ100V 10415.2 TURBOSLONG
1HZ10V 9999.98 TURBOSLONG
1HZ50V 10092.61 TURBOSSHORT
R_10 9990.805 TURBOSSHORT
R_25 10080.457 TURBOSLONG
1HZ100V 10412.78 TURBOSSHORT
1HZ100V 10409.45 TURBOSLONG
R_100 9851.13 TURBOSLONG
1HZ50V 10091.16 TURBOSLONG
1HZ50V 10091.16 TURBOSSHORT
R_10 9990.302 TURBOSSHORT
1HZ75V 9897.42 TURBOSSHORT
R_10 9990.184 TURBOSSHORT
1HZ10V 9999.79 TURBOSSHORT
1HZ50V 10087.35 TURBOSLONG
1HZ10V 9999.87 TURBOSSHORT
R_10 9989.596 TURBOSSHORT
1HZ100V 10415.1 TURBOSLONG
1HZ25V 10109.14 TURBOSSHORT
1HZ10V 10000.33 TURBOSSHORT
1HZ75V 9903.6 TURBOSLONG
R_75 9965.0385 TURBOSLONG
1HZ25V 10109.64 TURBOSSHORT
1HZ75V 9907.72 TURBOSLONG
1HZ25V 10109.84 TURBOSSHORT
1HZ75V 9907.92 TURBOSSHORT
1HZ75V 9903.74 TURBOSSHORT
1HZ50V 10095.35 TURBOSSHORT
1HZ10V 10000.22 TURBOSLONG
R_50 10065.1162 TURBOSLONG
1HZ25V 10112.47 TURBOSLONG
1HZ75V 9907.06 TURBOSSHORT
R_25

In [141]:

# Create an empty DataFrame to store the results
result_df = pd.DataFrame(columns=['symbol', 'calc_barrier', 'up_barrier', 'low_barrier', 'is_within_range'])

# Iterate over each row in the dataframe
for index, row in df_barrier.iterrows():
    calc_barriers = ast.literal_eval(row['calc_barrier_choices'])
    up_barriers = ast.literal_eval(row['up_barriers'])
    low_barriers = ast.literal_eval(row['low_barriers'])
    
    # Check if each barrier in calc_barrier_choices is within the range of corresponding up_barrier and low_barrier
    for i in range(len(calc_barriers)):
        calc_barrier = float(calc_barriers[i])
        up_barrier = float(up_barriers[i])
        low_barrier = float(low_barriers[i])
        
        # Store the result in a dictionary
        result = {
            'symbol': row['symbol'],
            'spot_price': row['spot'],
            'strike_choices': i + 1,  # Add 1 to the index to start from 1 instead of 0
            'calc_barrier': calc_barrier,
            'up_barrier': up_barrier,
            'low_barrier': low_barrier,
            'is_within_range': abs(calc_barrier) >= abs(low_barrier) and abs(calc_barrier) <= abs(up_barrier)
        }
        
        
        # Append the result to the result DataFrame
        result_df =  pd.concat([result_df, pd.DataFrame(result, index=[0])])

In [142]:
result_df[result_df["is_within_range"] == True]

,symbol,calc_barrier,up_barrier,low_barrier,is_within_range,spot_price,strike_choices
0,R_10,3.082,3.086,3.077,True,9989.746,1.0
0,R_10,4.547,4.554,4.540,True,9989.746,2.0
0,R_10,6.708,6.719,6.698,True,9989.746,3.0
0,R_10,9.898,9.913,9.883,True,9989.746,4.0
0,R_10,14.604,14.626,14.582,True,9989.746,5.0
...,...,...,...,...,...,...,...
0,R_25,1290.551,1295.426,1285.676,True,10087.377,16.0
0,R_25,1814.548,1821.403,1807.694,True,10087.377,17.0
0,R_25,2551.301,2560.939,2541.664,True,10087.377,18.0
0,R_25,3587.195,3600.746,3573.645,True,10087.377,19.0


## Validate the duration

In [143]:

# Define a dictionary to map the duration units to their corresponding values in seconds
duration_unit_mapping = {
    'd': 24 * 60 * 60,   # days to seconds
    'h': 60 * 60,        # hours to seconds
    'm': 60,             # minutes to seconds
    's': 1,              # seconds to seconds
    't': 1        # ticks to seconds
}

duration_unit_mapping_R = {
    'd': 24 * 60 * 60,   # days to seconds
    'h': 60 * 60,        # hours to seconds
    'm': 60,             # minutes to seconds
    's': 1,              # seconds to seconds
    't': 2        # ticks to seconds
}

# Convert date_expiry and date_start columns to datetime
df_duration['date_expiry'] = pd.to_datetime(df_duration['date_expiry'], unit='s')
df_duration['date_start'] = pd.to_datetime(df_duration['date_start'], unit='s')

# Calculate the duration based on the duration unit
df_duration['calculated_duration'] = np.where(
    df_duration['duration_unit'] == 'd',
    (df_duration['date_expiry'] - df_duration['date_start']).dt.days,
    np.where(df_duration["symbol"].str.startswith("R") ,
    (df_duration['date_expiry'] - df_duration['date_start']).dt.total_seconds() / df_duration['duration_unit'].map(duration_unit_mapping_R),
    (df_duration['date_expiry'] - df_duration['date_start']).dt.total_seconds() / df_duration['duration_unit'].map(duration_unit_mapping)
)
)

# Compare the calculated duration with the duration column
df_duration['duration_match'] = df_duration['calculated_duration'] == df_duration['duration']

# Print the DataFrame to see the results
df_duration[df_duration["duration_match"] == True]

,symbol,spot,duration,duration_unit,contract_type,date_expiry,date_start,calculated_duration,duration_match
0,R_10,9989.7460,388,m,TURBOSSHORT,2023-09-12 15:12:22,2023-09-12 08:44:22,388.0,True
1,R_25,10079.9590,28327,s,TURBOSLONG,2023-09-12 16:36:30,2023-09-12 08:44:23,28327.0,True
2,R_50,10056.7846,6,t,TURBOSSHORT,2023-09-12 08:44:35,2023-09-12 08:44:23,6.0,True
3,1HZ100V,10420.1400,152,m,TURBOSLONG,2023-09-12 11:16:24,2023-09-12 08:44:24,152.0,True
4,1HZ10V,9999.7900,301,d,TURBOSLONG,2024-07-09 23:59:59,2023-09-12 08:44:27,301.0,True
...,...,...,...,...,...,...,...,...,...
168,R_10,9992.6120,366,m,TURBOSSHORT,2023-09-12 14:54:53,2023-09-12 08:48:53,366.0,True
169,R_10,9992.3840,16,h,TURBOSSHORT,2023-09-13 00:48:54,2023-09-12 08:48:54,16.0,True
170,1HZ10V,10003.4500,19,h,TURBOSLONG,2023-09-13 03:48:55,2023-09-12 08:48:55,19.0,True
171,1HZ25V,10107.8000,975,m,TURBOSLONG,2023-09-13 01:03:58,2023-09-12 08:48:58,975.0,True


## Validate the min/max stake

The final min stake client can place: 

$ \text{min stake} = \text{max (USD 0.35, min stake calculate from formula b)} $

In [144]:
def max_stake(ul, spot, b):
    N_max = float(asset[asset["Asset"] == f"{ul}"]["Max Mul"]) * float(asset[asset["Asset"] == f"{ul}"]["Max Stake"]) / spot

    return N_max * abs (spot - b)

def min_stake(ul, spot, b):
    N_min = float(asset[asset["Asset"] == f"{ul}"]["Min Mul"]) * float(asset[asset["Asset"] == f"{ul}"]["Min Stake"]) / spot

    return N_min * abs (spot - b)

In [145]:
barriers = []
calc_max_stakes = []
calc_min_stakes = []

for row in df_stake.iterrows():
    barrier = ast.literal_eval(df_stake["barrier_choices"][row[0]])[1]
    barriers.append(str(barrier))
    calc_max_stake = max_stake(df_stake['symbol'][row[0]], df_stake['spot'][row[0]], df_stake['spot'][row[0]] + float(barrier))
    calc_max_stakes.append(calc_max_stake)
    calc_min_stake = min_stake(df_stake['symbol'][row[0]], df_stake['spot'][row[0]], df_stake['spot'][row[0]] + float(barrier))
    calc_min_stakes.append(max(0.35,calc_min_stake))

df_stake["barrier"] = barriers
df_stake["calc_max_stake"] = calc_max_stakes
df_stake["calc_min_stake"] = calc_min_stakes

In [146]:
# Check max_stake vs calc_max_stake & min
df_stake['max_stake_match'] = df_stake['max_stake'] == df_stake['calc_max_stake'].round(2)
df_stake['min_stake_match'] = df_stake['min_stake'] == df_stake['calc_min_stake'].round(2)

# Print the results
df_stake[(df_stake['min_stake_match'] == True) | (df_stake['max_stake_match'] == True )]

,symbol,spot,duration,duration_unit,contract_type,stake,barrier_choices,min_stake,max_stake,barrier,calc_max_stake,calc_min_stake,max_stake_match,min_stake_match
0,R_10,9989.7460,388,m,TURBOSSHORT,38.0,"['+3.082', '+4.547', '+6.709', '+9.899', '+14....",0.35,910.33,+4.547,910.333456,0.35,True,True
1,R_25,10079.9590,28327,s,TURBOSLONG,537.0,"['-7.775', '-10.932', '-15.371', '-21.611', '-...",0.35,1084.53,-10.932,1084.528221,0.35,True,True
2,R_50,10056.7846,6,t,TURBOSSHORT,653.0,"['+15.4893', '+20.9982', '+28.4663', '+38.5905...",0.35,835.19,+20.9982,835.185433,0.35,True,True
3,1HZ100V,10420.1400,152,m,TURBOSLONG,362.0,"['-32.14', '-42.01', '-54.9', '-71.76', '-93.8...",0.35,806.32,-42.01,806.323140,0.35,True,True
4,1HZ10V,9999.7900,301,d,TURBOSLONG,677.0,"['-3.09', '-4.55', '-6.71', '-9.91', '-14.61',...",0.35,910.02,-4.55,910.019110,0.35,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,R_10,9992.6120,366,m,TURBOSSHORT,47.0,"['+3.082', '+4.547', '+6.709', '+9.899', '+14....",0.35,910.07,+4.547,910.072361,0.35,True,True
169,R_10,9992.3840,16,h,TURBOSSHORT,755.0,"['+3.082', '+4.547', '+6.709', '+9.899', '+14....",0.35,910.09,+4.547,910.093127,0.35,True,True
170,1HZ10V,10003.4500,19,h,TURBOSLONG,644.0,"['-3.09', '-4.55', '-6.71', '-9.91', '-14.61',...",0.35,909.69,-4.55,909.686158,0.35,True,True
171,1HZ25V,10107.8000,975,m,TURBOSLONG,857.0,"['-7.78', '-10.93', '-15.37', '-21.61', '-30.3...",0.35,1081.34,-10.93,1081.343121,0.35,True,True


## Validate deriv's comission

$$ \text{contract price} = \frac{\text{stake}}{\text{number of contracts}} $$

premium for 1 contract:

If Long Turbo:

$$ \text{Premium 1 contract}=\text{BS}_{Ask}\text{(Long, S0, B, T)} $$

If Short Turbo:

$$ \text{Premium 1 contract}=\text{BS}_{Bid}\text{(Short, S0, B, T)} $$

where:
 
$$ \text{BS}_{Ask}\text{(Long, S0, B, T)} = \text{Ask} - \text{B} $$

$$ \text{BS}_{Bid}\text{(Long, S0, B, T)} = \text{B} - \text{Bid} $$

with:

$$ \text{Ask} = S_{t}*(1 + \text{Ask Spread}) $$
$$ \text{Bid} = S_{t}*(1 - \text{Bid Spread}) $$

In [147]:
df_ppp["api_premium"] = df_ppp['stake'] / df_ppp['number_of_contracts']

barriers = []


for row in df_stake.iterrows():
    barrier = ast.literal_eval(df_ppp["barrier_choices"][row[0]])[1]
    barriers.append(str(barrier))


df_ppp["barrier"] = barriers
df_ppp

,symbol,spot,duration,duration_unit,contract_type,stake,barrier_choices,min_stake,max_stake,number_of_contracts,api_premium,barrier
0,R_10,9989.7460,388,m,TURBOSSHORT,38.0,"['+3.082', '+4.547', '+6.709', '+9.899', '+14....",0.35,910.33,7.96075,4.773420,+4.547
1,R_25,10079.9590,28327,s,TURBOSLONG,537.0,"['-7.775', '-10.932', '-15.371', '-21.611', '-...",0.35,1084.53,46.68285,11.503154,-10.932
2,R_50,10056.7846,6,t,TURBOSSHORT,653.0,"['+15.4893', '+20.9982', '+28.4663', '+38.5905...",0.35,835.19,29.49695,22.137882,+20.9982
3,1HZ100V,10420.1400,152,m,TURBOSLONG,362.0,"['-32.14', '-42.01', '-54.9', '-71.76', '-93.8...",0.35,806.32,8.28755,43.679978,-42.01
4,1HZ10V,9999.7900,301,d,TURBOSLONG,677.0,"['-3.09', '-4.55', '-6.71', '-9.91', '-14.61',...",0.35,910.02,143.72874,4.710262,-4.55
...,...,...,...,...,...,...,...,...,...,...,...,...
168,R_10,9992.6120,366,m,TURBOSSHORT,47.0,"['+3.082', '+4.547', '+6.709', '+9.899', '+14....",0.35,910.07,9.84606,4.773483,+4.547
169,R_10,9992.3840,16,h,TURBOSSHORT,755.0,"['+3.082', '+4.547', '+6.709', '+9.899', '+14....",0.35,910.09,158.16564,4.773477,+4.547
170,1HZ10V,10003.4500,19,h,TURBOSLONG,644.0,"['-3.09', '-4.55', '-6.71', '-9.91', '-14.61',...",0.35,909.69,136.72106,4.710320,-4.55
171,1HZ25V,10107.8000,975,m,TURBOSLONG,857.0,"['-7.78', '-10.93', '-15.37', '-21.61', '-30.3...",0.35,1081.34,75.60665,11.334982,-10.93


In [233]:
def comm(spot, ul, barrier, contract_type):
    if ul == "R_75":
        spread = float(asset[asset["Asset"] == f"{ul}"]["Tick Size"]) * 1.1
    else:
        spread = float(asset[asset["Asset"] == f"{ul}"]["Tick Size"]) * 0.9

    if contract_type == "TURBOSLONG":
        return  spot * (1 + spread) - barrier
    else:
        return barrier - spot * (1 - spread) 

calc_comm = []

for row in df_ppp.iterrows():
    comms = comm(abs(float(df_ppp['spot'][row[0]])), df_ppp['symbol'][row[0]], 
                 float(df_ppp["spot"][row[0]]) + float(df_ppp["barrier"][row[0]]), 
                 str(df_ppp['contract_type'][row[0]]))

    calc_comm.append(float(comms))


df_ppp["calc_premium"] = calc_comm
df_ppp['perc_diff'] = (df_ppp['api_premium'] - df_ppp['calc_premium'])/df_ppp['calc_premium'] * 100
df_ppp[abs(df_ppp['perc_diff']) > 0.01]


,symbol,spot,duration,duration_unit,contract_type,stake,barrier_choices,min_stake,max_stake,number_of_contracts,api_premium,barrier,calc_premium,perc_diff


# Validate Price Proposal: Open Contract

Define all the necessary functon

In [478]:
def buy_contract(amount, symbol, contract_type, duration_unit):
    barriers = get_barrier_valid(amount, symbol, contract_type, duration_unit)
    request = {
            "buy": "1",
            "parameters": {
                "amount": amount,
                "barrier": barriers,
                "basis": "stake",
                "contract_type": contract_type,
                "currency": "USD",
                "duration": get_rand_duration(duration_unit),
                "duration_unit": duration_unit,
                "symbol": symbol
            },
            "price": amount
            }
    connection.send(json.dumps(request))

    res = connection.recv()
    if res is None:
        print("error")
    data = json.loads(res)
    if "error" in data:
        print("Error:", data)
        
    elif data["msg_type"] == "buy":
        print(data)

    return data

def sell_contract(id):
    request = {
            "sell": f"{id}",
            "price": 0
            }
    connection.send(json.dumps(request))

    res = connection.recv()
    if res is None:
        print("error")
    data = json.loads(res)
    if "error" in data:
        print("Error:", data)
        
    elif data["msg_type"] == "sell":
        print(data)

def stream_open_contract():
    request_open_contract={
        "proposal_open_contract": 1,
        "subscribe": 1
    }
    connection.send(json.dumps(request_open_contract))
    while True:
        res = connection.recv()
        if res is None:
            break
        data = json.loads(res)
        if "error" in data:
            print("Error:", data["error"]["message"])
            connection.close()
            break
        elif data["msg_type"] == "proposal_open_contract":  
            print(data['proposal_open_contract'])
            if 'audit_details' in data['proposal_open_contract']:
                return data['proposal_open_contract']['contract_id']
                break

def get_open_contract(duration): #to stream n get all the current open contract
    start_time = time.time()
    request_open_contract={
        "proposal_open_contract": 1,
        "subscribe": 1
    }
    connection.send(json.dumps(request_open_contract))
    datums=[]
    while time.time() - start_time < duration:
        res = connection.recv()
        if res is None:
            break
        data = json.loads(res)
        if "error" in data:
            print("Error:", data["error"]["message"])
            connection.close()
            break
        elif data["msg_type"] == "proposal_open_contract":
            if len(data['proposal_open_contract']) == 0:
                print("no open contract")
                connection.close()
                break
            else:
                datum = {
                        "barrier": data['proposal_open_contract']["barrier"],
                        "bid_price": data['proposal_open_contract']["bid_price"],
                        "buy_price": data['proposal_open_contract']["buy_price"],
                        "current_spot": data['proposal_open_contract']["current_spot"],
                        "entry_spot": data['proposal_open_contract']["entry_spot"],
                        "number_of_contracts": data['proposal_open_contract']["number_of_contracts"],
                        "contract_id": data['proposal_open_contract']["contract_id"],
                        "contract_type": data['proposal_open_contract']["contract_type"],
                        "underlying": data['proposal_open_contract']["underlying"]}
                datums.append(datum)
                print(data['proposal_open_contract'])
    print(datums)
    df =pd.DataFrame(datums)
    return df

def get_contract(id): #to get contract by id 
    request_open_contract={
        "proposal_open_contract": 1,
        "contract_id": id 
    }
    connection.send(json.dumps(request_open_contract))
    res = connection.recv()
    if res is None:
        print("error")
    else:
        try:
            data = json.loads(res)
            if "error" in data:
                print("Error:", data["error"]["message"])
                
            elif data["msg_type"] == "proposal_open_contract":
                return data
        except json.decoder.JSONDecodeError:    
            print('The string does NOT contain valid JSON')
        
        
            
            

# Check Sell Price when manually sell the contract

#### Need to buy N number of contracts

- buy 200 contracts (random sampling)
- sell everything
- get contract details and calculate sell price
- compare with sell_price in the API

In [485]:
sell_contract_ids = []

buy 200 contracts
- save the contract id to get the detail later

In [486]:
for i in range (200):
    stake = random.choice(amount)
    contract_type = random.choice(contract_types)
    duration_unit = random.choice(duration_units)
    underlying = random.choice(indices)

    connection = websocket.create_connection(ws)
    auth_token(api_token)
    sell_contract_ids.append(buy_contract(stake, underlying, contract_type, duration_unit))

    buy_contract(stake, underlying, contract_type, duration_unit)
    connection.close()


{'account_list': [{'account_category': 'trading', 'account_type': 'binary', 'created_at': 1694583451, 'currency': 'USD', 'is_disabled': 0, 'is_virtual': 0, 'landing_company_name': 'svg', 'linked_to': [], 'loginid': 'CR90000013'}, {'account_category': 'trading', 'account_type': 'binary', 'created_at': 1694583450, 'currency': 'USD', 'is_disabled': 0, 'is_virtual': 1, 'landing_company_name': 'virtual', 'linked_to': [], 'loginid': 'VRTC90000013'}], 'balance': 12177.14, 'country': 'aq', 'currency': 'USD', 'email': 'onadwdawk@gmafdicl.com', 'fullname': '  ', 'is_virtual': 1, 'landing_company_fullname': 'Deriv Limited', 'landing_company_name': 'virtual', 'linked_to': [], 'local_currencies': {'AAD': {'fractional_digits': 2}}, 'loginid': 'VRTC90000013', 'preferred_language': 'EN', 'scopes': ['read', 'trade', 'payments', 'admin'], 'upgradeable_landing_companies': ['svg'], 'user_id': 14}
{'buy': {'balance_after': 12171.14, 'buy_price': 6, 'contract_id': 16619, 'longcode': 'Your payout will grow b

In [533]:
sell_contract_ids
df_data = {
    'contract_id': [],
    'buy_price': [],
    'contract_type': []
}
for item in sell_contract_ids:
    buy_info = item.get('buy', {})
    contract_info = item.get('echo_req', {}).get('parameters', {})  # Get the 'buy' dictionary or an empty dictionary if it doesn't exist
    df_data['contract_id'].append(buy_info.get('contract_id', None))
    df_data['buy_price'].append(buy_info.get('buy_price', None))
    df_data['contract_type'].append(contract_info.get('contract_type', None))
    print (item)



df = pd.DataFrame(df_data)
df = df.dropna()
df

{'buy': {'balance_after': 12171.14, 'buy_price': 6, 'contract_id': 16619, 'longcode': 'Your payout will grow by 0.01070 for every point below the barrier at the expiry time if the barrier is not touched during the contract duration. You will start making a profit when the payout is higher than your stake.', 'payout': 0, 'purchase_time': 1695049706, 'shortcode': 'TURBOSSHORT_R_75_6.00_1695049706_1725753599_S5249610P_0.01070_1695049706', 'start_time': 1695049706, 'transaction_id': 29779}, 'echo_req': {'buy': '1', 'parameters': {'amount': 6, 'app_markup_percentage': '0', 'barrier': '+524.961', 'basis': 'stake', 'contract_type': 'TURBOSSHORT', 'currency': 'USD', 'duration': 355, 'duration_unit': 'd', 'symbol': 'R_75'}, 'price': 6}, 'msg_type': 'buy'}
{'buy': {'balance_after': 12164.14, 'buy_price': 1, 'contract_id': 16659, 'longcode': 'Your payout will grow by 0.23794 for every point above the barrier at the expiry time if the barrier is not touched during the contract duration. You will s

,contract_id,buy_price,contract_type
0,16619.0,6.0,TURBOSSHORT
1,16659.0,1.0,TURBOSLONG
2,16699.0,5.0,TURBOSSHORT
3,16739.0,7.0,TURBOSSHORT
4,16779.0,4.0,TURBOSSHORT
5,16819.0,7.0,TURBOSLONG
6,16859.0,1.0,TURBOSLONG
7,16899.0,6.0,TURBOSSHORT
8,16939.0,3.0,TURBOSSHORT
9,16979.0,7.0,TURBOSLONG


Just in case you want to see the current open contracts, run the script below

In [523]:
connection = websocket.create_connection(ws)
auth_token(api_token)
df_poc = get_open_contract(1)
connection.close()

df_poc

{'account_list': [{'account_category': 'trading', 'account_type': 'binary', 'created_at': 1694583451, 'currency': 'USD', 'is_disabled': 0, 'is_virtual': 0, 'landing_company_name': 'svg', 'linked_to': [], 'loginid': 'CR90000013'}, {'account_category': 'trading', 'account_type': 'binary', 'created_at': 1694583450, 'currency': 'USD', 'is_disabled': 0, 'is_virtual': 1, 'landing_company_name': 'virtual', 'linked_to': [], 'loginid': 'VRTC90000013'}], 'balance': 12163.35, 'country': 'aq', 'currency': 'USD', 'email': 'onadwdawk@gmafdicl.com', 'fullname': '  ', 'is_virtual': 1, 'landing_company_fullname': 'Deriv Limited', 'landing_company_name': 'virtual', 'linked_to': [], 'local_currencies': {'AAD': {'fractional_digits': 2}}, 'loginid': 'VRTC90000013', 'preferred_language': 'EN', 'scopes': ['read', 'trade', 'payments', 'admin'], 'upgradeable_landing_companies': ['svg'], 'user_id': 14}
no open contract
[]


""


sell all the contracts

In [513]:
connection = websocket.create_connection(ws)
auth_token(api_token)

df=df_poc.dropna()

for i in df.iterrows():
        sell_contract(int(i[1]["contract_id"]))
        time.sleep(1)

{'account_list': [{'account_category': 'trading', 'account_type': 'binary', 'created_at': 1694583451, 'currency': 'USD', 'is_disabled': 0, 'is_virtual': 0, 'landing_company_name': 'svg', 'linked_to': [], 'loginid': 'CR90000013'}, {'account_category': 'trading', 'account_type': 'binary', 'created_at': 1694583450, 'currency': 'USD', 'is_disabled': 0, 'is_virtual': 1, 'landing_company_name': 'virtual', 'linked_to': [], 'loginid': 'VRTC90000013'}], 'balance': 12141.91, 'country': 'aq', 'currency': 'USD', 'email': 'onadwdawk@gmafdicl.com', 'fullname': '  ', 'is_virtual': 1, 'landing_company_fullname': 'Deriv Limited', 'landing_company_name': 'virtual', 'linked_to': [], 'local_currencies': {'AAD': {'fractional_digits': 2}}, 'loginid': 'VRTC90000013', 'preferred_language': 'EN', 'scopes': ['read', 'trade', 'payments', 'admin'], 'upgradeable_landing_companies': ['svg'], 'user_id': 14}
{'echo_req': {'price': 0, 'sell': '17679'}, 'msg_type': 'sell', 'sell': {'balance_after': 12163.35, 'contract_

Get the contract details using the contract ids

In [541]:
datums = []

for i in df.iterrows():
    time.sleep(1)
    connection = websocket.create_connection(ws)
    auth_token(api_token)

    if (np.isnan(i[1]['contract_id'])):
        next
    data = get_contract(int(i[1]["contract_id"]))
    if (data is not None):     
        if ('exit_tick' in data['proposal_open_contract']):
            if (data is None):
                next
            datum = {
                        "barrier": data['proposal_open_contract']["barrier"],
                        "bid_price": data['proposal_open_contract']["bid_price"],
                        "buy_price": data['proposal_open_contract']["buy_price"],
                        "current_spot": data['proposal_open_contract']["current_spot"],
                        "entry_spot": data['proposal_open_contract']["entry_spot"],
                        "number_of_contracts": data['proposal_open_contract']["number_of_contracts"],
                        "contract_id": data['proposal_open_contract']["contract_id"],
                        "contract_type": data['proposal_open_contract']["contract_type"],
                        "underlying": data['proposal_open_contract']["underlying"],
                        "date_start": data['proposal_open_contract']["date_start"],
                        "date_expiry": data['proposal_open_contract']["date_expiry"],
                        "date_settlement": data['proposal_open_contract']["date_settlement"],
                        "exit_tick": data['proposal_open_contract']["exit_tick"],
                        "entry_tick_display_value": data['proposal_open_contract']["entry_tick_display_value"],
                        "entry_tick_time": data['proposal_open_contract']["entry_tick_time"],
                        "exit_tick_time": data['proposal_open_contract']["exit_tick_time"],
                        "expiry_time": data['proposal_open_contract']["expiry_time"],
                        "sell_price": data['proposal_open_contract']["sell_price"],
                        "sell_spot": data['proposal_open_contract']["sell_spot"]
                    }
        else:
            datum = {
                        "barrier": data['proposal_open_contract']["barrier"],
                        "bid_price": data['proposal_open_contract']["bid_price"],
                        "buy_price": data['proposal_open_contract']["buy_price"],
                        "current_spot": data['proposal_open_contract']["current_spot"],
                        "entry_spot": data['proposal_open_contract']["entry_spot"],
                        "number_of_contracts": data['proposal_open_contract']["number_of_contracts"],
                        "contract_id": data['proposal_open_contract']["contract_id"],
                        "contract_type": data['proposal_open_contract']["contract_type"],
                        "underlying": data['proposal_open_contract']["underlying"],
                        "date_start": data['proposal_open_contract']["date_start"],
                        "date_expiry": data['proposal_open_contract']["date_expiry"],
                        "date_settlement": data['proposal_open_contract']["date_settlement"],
                        "entry_tick_display_value": data['proposal_open_contract']["entry_tick_display_value"],
                        "entry_tick_time": data['proposal_open_contract']["entry_tick_time"],
                        "expiry_time": data['proposal_open_contract']["expiry_time"],
                    }


    datums.append(datum)
df_poc1 =pd.DataFrame(datums)
    

{'account_list': [{'account_category': 'trading', 'account_type': 'binary', 'created_at': 1694583451, 'currency': 'USD', 'is_disabled': 0, 'is_virtual': 0, 'landing_company_name': 'svg', 'linked_to': [], 'loginid': 'CR90000013'}, {'account_category': 'trading', 'account_type': 'binary', 'created_at': 1694583450, 'currency': 'USD', 'is_disabled': 0, 'is_virtual': 1, 'landing_company_name': 'virtual', 'linked_to': [], 'loginid': 'VRTC90000013'}], 'balance': 12163.35, 'country': 'aq', 'currency': 'USD', 'email': 'onadwdawk@gmafdicl.com', 'fullname': '  ', 'is_virtual': 1, 'landing_company_fullname': 'Deriv Limited', 'landing_company_name': 'virtual', 'linked_to': [], 'local_currencies': {'AAD': {'fractional_digits': 2}}, 'loginid': 'VRTC90000013', 'preferred_language': 'EN', 'scopes': ['read', 'trade', 'payments', 'admin'], 'upgradeable_landing_companies': ['svg'], 'user_id': 14}
{'account_list': [{'account_category': 'trading', 'account_type': 'binary', 'created_at': 1694583451, 'currenc

In [542]:
df_poc1

,barrier,bid_price,buy_price,current_spot,entry_spot,number_of_contracts,contract_id,contract_type,underlying,date_start,date_expiry,date_settlement,exit_tick,entry_tick_display_value,entry_tick_time,exit_tick_time,expiry_time,sell_price,sell_spot
0,171591.5785,0.00,6,174158.7247,171066.6175,0.01070,16619,TURBOSSHORT,R_75,1695049706,1725753599,1725753600,171623.1680,171066.6175,1695049706,1695050150,1725753599,0.00,171623.1680
1,8899.31,3.39,1,8918.2600,8903.3700,0.23794,16659,TURBOSLONG,1HZ10V,1695049707,1695092817,1695092817,8913.6900,8903.37,1695049707,1695050274,1695092817,3.39,8913.6900
2,1959.654,8.26,5,1943.0770,1957.5310,2.23822,16699,TURBOSSHORT,R_25,1695049709,1695053202,1695053202,1955.8520,1957.531,1695049708,1695050274,1695053202,8.26,1955.8520
3,8907.46,0.00,7,8918.2400,8903.4000,1.66560,16739,TURBOSSHORT,1HZ10V,1695049710,1695075646,1695075646,8907.5200,8903.40,1695049710,1695049932,1695075646,0.00,8907.5200
4,1872.30,2.91,4,1879.9800,1864.7800,0.50361,16779,TURBOSSHORT,R_100,1695049711,1695049728,1695049728,1866.3600,1864.78,1695049710,1695049730,1695049728,2.91,1866.3600
5,1857.19,4.16,7,1879.1800,1864.7100,0.88132,16819,TURBOSLONG,R_100,1695049713,1695082113,1695082113,1862.3300,1864.71,1695049712,1695050276,1695082113,4.16,1862.3300
6,1857.28,0.58,1,1879.1800,1864.8000,0.12590,16859,TURBOSLONG,R_100,1695049714,1695104287,1695104287,1862.2700,1864.80,1695049714,1695050274,1695104287,0.58,1862.2700
7,437873.30,0.00,6,437941.4900,437399.0900,0.01220,16899,TURBOSSHORT,1HZ25V,1695049716,1695060516,1695060516,437879.0900,437399.09,1695049716,1695049996,1695060516,0.00,437879.0900
8,8907.71,2.74,3,8918.6500,8903.6500,0.71383,16939,TURBOSSHORT,1HZ10V,1695049717,1695049725,1695049725,8903.8600,8903.65,1695049717,1695049726,1695049725,2.74,8903.8600
9,636216.45,0.00,7,635153.9200,637549.3300,0.00506,16979,TURBOSLONG,1HZ50V,1695049718,1695056918,1695056918,636198.1300,637549.33,1695049718,1695050138,1695056918,0.00,636198.1300


Check for the sell_price and compare with the calculated price

In [543]:

def bid(spot, ul, barrier, contract_type):
    if ul == "R_75":
        spread = float(asset[asset["Asset"] == f"{ul}"]["Tick Size"]) * 1.1
    else:
        spread = float(asset[asset["Asset"] == f"{ul}"]["Tick Size"]) * 0.9    
    if contract_type == "TURBOSLONG":
        return  max(0,spot * (1 - spread) - barrier)
    else:
        return max(0,barrier - spot * (1 + spread)  )     

calc_comm = []


for row in df_poc1.iterrows():
    comms = bid(abs(float(df_poc1['exit_tick'][row[0]])),
                 str(df_poc1['underlying'][row[0]]), 
                 float(df_poc1["barrier"][row[0]]), 
                 str(df_poc1['contract_type'][row[0]]))

    calc_comm.append(float(comms))


df_poc1["calc_sell_price_1contract"] = calc_comm 
df_poc1["calc_sell_price"] = df_poc1["calc_sell_price_1contract"] * df_poc1[ "number_of_contracts"]
df_poc1 ["is_same"] = df_poc1["calc_sell_price"].round(2) == df_poc1["sell_price"]
df_poc1["diff"] = abs(df_poc1["calc_sell_price"] -  df_poc1["sell_price"])/df_poc1["calc_sell_price"] * 100
df_poc1["is_ok"] = df_poc1['diff'] < 0.1



df_poc1.to_csv("manual_sell_price.csv")


# Check Sell Price at expiry

#### Need to buy N number of contracts

- buy 200 contracts (random sampling)
- wait til expiry
- get contract details and calculate sell price
- compare with sell_price in the API

In [391]:
contract_ids=[]

buy contracts and save the contract id data frame

In [430]:
connection = websocket.create_connection(ws)
auth_token(api_token)
for i in range (50):
    stake = random.choice(amount)
    contract_type = random.choice(contract_types)
    duration_unit = random.choice(["m"])
    underlying = random.choice(indices)

    
    contract_ids.append(buy_contract(stake, underlying, contract_type, duration_unit))
    time.sleep(2.5)
connection.close()




{'account_list': [{'account_category': 'trading', 'account_type': 'binary', 'created_at': 1694583451, 'currency': 'USD', 'is_disabled': 0, 'is_virtual': 0, 'landing_company_name': 'svg', 'linked_to': [], 'loginid': 'CR90000013'}, {'account_category': 'trading', 'account_type': 'binary', 'created_at': 1694583450, 'currency': 'USD', 'is_disabled': 0, 'is_virtual': 1, 'landing_company_name': 'virtual', 'linked_to': [], 'loginid': 'VRTC90000013'}], 'balance': 12142.25, 'country': 'aq', 'currency': 'USD', 'email': 'onadwdawk@gmafdicl.com', 'fullname': '  ', 'is_virtual': 1, 'landing_company_fullname': 'Deriv Limited', 'landing_company_name': 'virtual', 'linked_to': [], 'local_currencies': {'AAD': {'fractional_digits': 2}}, 'loginid': 'VRTC90000013', 'preferred_language': 'EN', 'scopes': ['read', 'trade', 'payments', 'admin'], 'upgradeable_landing_companies': ['svg'], 'user_id': 14}
{'buy': {'balance_after': 12134.25, 'buy_price': 8, 'contract_id': 15659, 'longcode': 'Your payout will grow b

In [431]:
contract_ids
df_data = {
    'contract_id': [],
    'buy_price': [],
    'contract_type': []
}
for item in contract_ids:
    buy_info = item.get('buy', {})
    contract_info = item.get('echo_req', {}).get('parameters', {})  # Get the 'buy' dictionary or an empty dictionary if it doesn't exist
    df_data['contract_id'].append(buy_info.get('contract_id', None))
    df_data['buy_price'].append(buy_info.get('buy_price', None))
    df_data['contract_type'].append(contract_info.get('contract_type', None))
    print (item)



df = pd.DataFrame(df_data)
df

{'buy': {'balance_after': 12885.42, 'buy_price': 2, 'contract_id': 12499, 'longcode': 'Your payout will grow by 0.00416 for every point above the barrier at the expiry time if the barrier is not touched during the contract duration. You will start making a profit when the payout is higher than your stake.', 'payout': 0, 'purchase_time': 1694768456, 'shortcode': 'TURBOSLONG_1HZ25V_2.00_1694768456_1694814056_S-46378P_0.00416_1694768456', 'start_time': 1694768456, 'transaction_id': 21979}, 'echo_req': {'buy': '1', 'parameters': {'amount': 2, 'app_markup_percentage': '0', 'barrier': '-463.78', 'basis': 'stake', 'contract_type': 'TURBOSLONG', 'currency': 'USD', 'duration': 760, 'duration_unit': 'm', 'symbol': '1HZ25V'}, 'price': 2}, 'msg_type': 'buy'}
{'buy': {'balance_after': 12880.42, 'buy_price': 5, 'contract_id': 12519, 'longcode': 'Your payout will grow by 0.01040 for every point above the barrier at the expiry time if the barrier is not touched during the contract duration. You will s

,contract_id,buy_price,contract_type
0,12499.0,2.0,TURBOSLONG
1,12519.0,5.0,TURBOSLONG
2,12539.0,6.0,TURBOSSHORT
3,12559.0,5.0,TURBOSSHORT
4,12579.0,6.0,TURBOSSHORT
...,...,...,...
205,NaN,NaN,TURBOSSHORT
206,16539.0,6.0,TURBOSSHORT
207,NaN,NaN,TURBOSSHORT
208,16579.0,2.0,TURBOSSHORT


get the contract detail using the saved contract id

In [480]:
datums = []

for i in df.iterrows():
    time.sleep(1)
    connection = websocket.create_connection(ws)
    auth_token(api_token)

    if (np.isnan(i[1]['contract_id'])):
        next
    data = get_contract(i[1]["contract_id"])
    if (data is not None):     
        if ('exit_tick' in data['proposal_open_contract']):
            if (data is None):
                next
            datum = {
                        "barrier": data['proposal_open_contract']["barrier"],
                        "bid_price": data['proposal_open_contract']["bid_price"],
                        "buy_price": data['proposal_open_contract']["buy_price"],
                        "current_spot": data['proposal_open_contract']["current_spot"],
                        "entry_spot": data['proposal_open_contract']["entry_spot"],
                        "number_of_contracts": data['proposal_open_contract']["number_of_contracts"],
                        "contract_id": data['proposal_open_contract']["contract_id"],
                        "contract_type": data['proposal_open_contract']["contract_type"],
                        "underlying": data['proposal_open_contract']["underlying"],
                        "date_start": data['proposal_open_contract']["date_start"],
                        "date_expiry": data['proposal_open_contract']["date_expiry"],
                        "date_settlement": data['proposal_open_contract']["date_settlement"],
                        "exit_tick": data['proposal_open_contract']["exit_tick"],
                        "entry_tick_display_value": data['proposal_open_contract']["entry_tick_display_value"],
                        "entry_tick_time": data['proposal_open_contract']["entry_tick_time"],
                        "exit_tick_time": data['proposal_open_contract']["exit_tick_time"],
                        "expiry_time": data['proposal_open_contract']["expiry_time"],
                        "sell_price": data['proposal_open_contract']["sell_price"],
                        "sell_spot": data['proposal_open_contract']["sell_spot"]
                    }
        else:
            datum = {
                        "barrier": data['proposal_open_contract']["barrier"],
                        "bid_price": data['proposal_open_contract']["bid_price"],
                        "buy_price": data['proposal_open_contract']["buy_price"],
                        "current_spot": data['proposal_open_contract']["current_spot"],
                        "entry_spot": data['proposal_open_contract']["entry_spot"],
                        "number_of_contracts": data['proposal_open_contract']["number_of_contracts"],
                        "contract_id": data['proposal_open_contract']["contract_id"],
                        "contract_type": data['proposal_open_contract']["contract_type"],
                        "underlying": data['proposal_open_contract']["underlying"],
                        "date_start": data['proposal_open_contract']["date_start"],
                        "date_expiry": data['proposal_open_contract']["date_expiry"],
                        "date_settlement": data['proposal_open_contract']["date_settlement"],
                        "entry_tick_display_value": data['proposal_open_contract']["entry_tick_display_value"],
                        "entry_tick_time": data['proposal_open_contract']["entry_tick_time"],
                        "expiry_time": data['proposal_open_contract']["expiry_time"],
                    }

    datums.append(datum)
    connection.close()
df_exp1 =pd.DataFrame(datums)
df_exp1

{'account_list': [{'account_category': 'trading', 'account_type': 'binary', 'created_at': 1694583451, 'currency': 'USD', 'is_disabled': 0, 'is_virtual': 0, 'landing_company_name': 'svg', 'linked_to': [], 'loginid': 'CR90000013'}, {'account_category': 'trading', 'account_type': 'binary', 'created_at': 1694583450, 'currency': 'USD', 'is_disabled': 0, 'is_virtual': 1, 'landing_company_name': 'virtual', 'linked_to': [], 'loginid': 'VRTC90000013'}], 'balance': 12177.14, 'country': 'aq', 'currency': 'USD', 'email': 'onadwdawk@gmafdicl.com', 'fullname': '  ', 'is_virtual': 1, 'landing_company_fullname': 'Deriv Limited', 'landing_company_name': 'virtual', 'linked_to': [], 'local_currencies': {'AAD': {'fractional_digits': 2}}, 'loginid': 'VRTC90000013', 'preferred_language': 'EN', 'scopes': ['read', 'trade', 'payments', 'admin'], 'upgradeable_landing_companies': ['svg'], 'user_id': 14}
{'account_list': [{'account_category': 'trading', 'account_type': 'binary', 'created_at': 1694583451, 'currenc

,barrier,bid_price,buy_price,current_spot,entry_spot,number_of_contracts,contract_id,contract_type,underlying,date_start,date_expiry,date_settlement,exit_tick,entry_tick_display_value,entry_tick_time,exit_tick_time,expiry_time,sell_price,sell_spot
0,426618.81,1.56,2,438571.510,427082.590,0.00416,12499,TURBOSLONG,1HZ25V,1694768456,1694814056,1694814056,427010.330,427082.59,1694768456,1694768577,1694814056,1.56,427010.330
1,426568.56,3.92,5,438604.120,427032.340,0.01040,12519,TURBOSLONG,1HZ25V,1694768460,1694803980,1694803980,426962.160,427032.34,1694768460,1694768574,1694803980,3.92,426962.160
2,7105.835,7.27,6,7154.305,7102.603,1.76836,12539,TURBOSSHORT,R_10,1694768463,1694813163,1694813163,7101.561,7102.603,1694768462,1694768570,1694813163,7.27,7101.561
3,8936.50,0.00,5,8897.320,8932.430,1.18676,12559,TURBOSSHORT,1HZ10V,1694768602,1694817622,1694817622,8936.540,8932.43,1694768602,1694770724,1694817622,0.00,8936.540
4,643490.44,0.00,6,648753.320,642155.640,0.00433,12579,TURBOSSHORT,1HZ50V,1694768605,1694847445,1694847445,643557.080,642155.64,1694768605,1694784624,1694847445,0.00,643557.080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,9167.48,0.00,3,8742.140,9139.400,0.10282,16499,TURBOSSHORT,1HZ75V,1694770501,1694812861,1694812861,9168.490,9139.40,1694770501,1694771099,1694812861,0.00,9168.490
206,9169.99,0.00,6,8742.110,9141.910,0.20563,16539,TURBOSSHORT,1HZ75V,1694770507,1694798587,1694798587,9171.360,9141.91,1694770507,1694771102,1694798587,0.00,9171.360
207,9169.99,0.00,6,8742.110,9141.910,0.20563,16539,TURBOSSHORT,1HZ75V,1694770507,1694798587,1694798587,9171.360,9141.91,1694770507,1694771102,1694798587,0.00,9171.360
208,1746.26,0.00,2,1884.580,1739.250,0.27012,16579,TURBOSSHORT,R_100,1694770513,1694822773,1694822773,1746.290,1739.25,1694770512,1694770794,1694822773,0.00,1746.290


Calculate the sell_price in the API with the calculated sell price

In [481]:
def calc_sell_price(x):
    def bid(spot, barrier, contract_type):   
        if contract_type == "TURBOSLONG":
            return  max(0,spot  - barrier)
        else:
            return max(0,barrier - spot )     

    calc_comm = []


    for row in x.iterrows():
        comms = bid(abs(float(x['exit_tick'][row[0]])), 
                    float(x["barrier"][row[0]]), 
                    str(x['contract_type'][row[0]]))

        calc_comm.append(float(comms))


    x["calc_sell_price_1contract"] = calc_comm 
    x["calc_sell_price"] = x["calc_sell_price_1contract"] * x[ "number_of_contracts"]
    x ["is_same"] = x["calc_sell_price"].round(2) == x["sell_price"]
    x["diff"] = abs(x["calc_sell_price"] -  x["sell_price"])/x["calc_sell_price"] * 100
    x["is_ok"] = x['diff'] < 0.1

    x #= x[(x['diff'].isna() == False) & (x['sell_price'].isna() == False)]

    return x



calc_sell_price(df_exp1)

,barrier,bid_price,buy_price,current_spot,entry_spot,number_of_contracts,contract_id,contract_type,underlying,date_start,...,entry_tick_time,exit_tick_time,expiry_time,sell_price,sell_spot,calc_sell_price_1contract,calc_sell_price,is_same,diff,is_ok
0,426618.81,1.56,2,438571.510,427082.590,0.00416,12499,TURBOSLONG,1HZ25V,1694768456,...,1694768456,1694768577,1694814056,1.56,427010.330,391.520,1.628723,False,4.219452,False
1,426568.56,3.92,5,438604.120,427032.340,0.01040,12519,TURBOSLONG,1HZ25V,1694768460,...,1694768460,1694768574,1694803980,3.92,426962.160,393.600,4.093440,False,4.237023,False
2,7105.835,7.27,6,7154.305,7102.603,1.76836,12539,TURBOSSHORT,R_10,1694768463,...,1694768462,1694768570,1694813163,7.27,7101.561,4.274,7.557971,False,3.810158,False
3,8936.50,0.00,5,8897.320,8932.430,1.18676,12559,TURBOSSHORT,1HZ10V,1694768602,...,1694768602,1694770724,1694817622,0.00,8936.540,0.000,0.000000,True,NaN,False
4,643490.44,0.00,6,648753.320,642155.640,0.00433,12579,TURBOSSHORT,1HZ50V,1694768605,...,1694768605,1694784624,1694847445,0.00,643557.080,0.000,0.000000,True,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,9167.48,0.00,3,8742.140,9139.400,0.10282,16499,TURBOSSHORT,1HZ75V,1694770501,...,1694770501,1694771099,1694812861,0.00,9168.490,0.000,0.000000,True,NaN,False
206,9169.99,0.00,6,8742.110,9141.910,0.20563,16539,TURBOSSHORT,1HZ75V,1694770507,...,1694770507,1694771102,1694798587,0.00,9171.360,0.000,0.000000,True,NaN,False
207,9169.99,0.00,6,8742.110,9141.910,0.20563,16539,TURBOSSHORT,1HZ75V,1694770507,...,1694770507,1694771102,1694798587,0.00,9171.360,0.000,0.000000,True,NaN,False
208,1746.26,0.00,2,1884.580,1739.250,0.27012,16579,TURBOSSHORT,R_100,1694770513,...,1694770512,1694770794,1694822773,0.00,1746.290,0.000,0.000000,True,NaN,False


In [484]:
df_result = calc_sell_price(df_exp1)

df_result.to_csv("expiry_price.csv")